    mistral_api_key="SDlJ6TyxuOseTYDmLrZYjXkxfP7qc33u"

In [1]:
import pandas as pd

df = pd.read_csv('dataset.csv')

# le top 20% des musiques les plus populaires
top_songs = df[df['popularity'] >= df['popularity'].quantile(0.8)]

# Calcul des paramètres optimaux (médianes)
optimal_params = top_songs[[
    'danceability', 'energy', 'key', 'loudness', 'mode',
    'speechiness', 'acousticness', 'instrumentalness',
    'liveness', 'valence', 'tempo', 'duration_ms'
]].median().to_dict()

print("PARAMÈTRES OPTIMAUX:")
for param, value in optimal_params.items():
    print(f"• {param}: {value:.3f}")

PARAMÈTRES OPTIMAUX:
• danceability: 0.597
• energy: 0.672
• key: 5.000
• loudness: -6.574
• mode: 1.000
• speechiness: 0.047
• acousticness: 0.154
• instrumentalness: 0.000
• liveness: 0.124
• valence: 0.451
• tempo: 120.146
• duration_ms: 210165.000


In [2]:
import pandas as pd

def get_optimal_parameters(genre=None):
    """Retourne les paramètres optimaux pour une musique populaire"""
    df = pd.read_csv('dataset.csv')
    
    if genre:
        # Filtre par genre spécifique
        genre_songs = df[df['track_genre'] == genre]
        if len(genre_songs) == 0:
            available_genres = df['track_genre'].unique()
            print(f"❌ Genre '{genre}' non trouvé. Genres disponibles: {list(available_genres)[:10]}...")
            return None
        
        # Top 20% du genre
        top_songs = genre_songs[genre_songs['popularity'] >= genre_songs['popularity'].quantile(0.8)]
        print(f"🎵 Paramètres optimaux pour le genre: {genre}")
    else:
        # Tous genres confondus
        top_songs = df[df['popularity'] >= df['popularity'].quantile(0.8)]
        print("🎵 Paramètres optimaux (tous genres)")
    
    # Calcul des paramètres optimaux
    optimal_params = top_songs[[
        'danceability', 'energy', 'key', 'loudness', 'mode',
        'speechiness', 'acousticness', 'instrumentalness',
        'liveness', 'valence', 'tempo', 'duration_ms'
    ]].median().to_dict()
    
    return optimal_params

def display_parameters(params):
    """Affiche les paramètres de manière lisible"""
    if params is None:
        return
        
    print("\n📊 PARAMÈTRES OPTIMAUX:")
    for param, value in params.items():
        if param in ['danceability', 'energy', 'valence']:
            print(f"• {param}: {value:.3f} ({(value*100):.0f}%)")
        elif param == 'loudness':
            print(f"• {param}: {value:.1f} dB")
        elif param == 'tempo':
            print(f"• {param}: {value:.0f} BPM")
        elif param == 'duration_ms':
            minutes = value / 60000
            print(f"• {param}: {minutes:.1f} min")
        else:
            print(f"• {param}: {value:.3f}")

# Exemples d'utilisation
print("=== IA GÉNÉRATIVE DE PARAMÈTRES MUSICAUX ===")

# 1. Sans contrainte
print("\n1. Sans contrainte de genre:")
params_all = get_optimal_parameters()
display_parameters(params_all)

# 2. Avec genre spécifique
print("\n2. Avec genre 'acoustic':")
params_acoustic = get_optimal_parameters('acoustic')
display_parameters(params_acoustic)

# 3. Avec un autre genre
print("\n3. Avec genre 'pop':")
params_pop = get_optimal_parameters('pop')
display_parameters(params_pop)

=== IA GÉNÉRATIVE DE PARAMÈTRES MUSICAUX ===

1. Sans contrainte de genre:
🎵 Paramètres optimaux (tous genres)

📊 PARAMÈTRES OPTIMAUX:
• danceability: 0.597 (60%)
• energy: 0.672 (67%)
• key: 5.000
• loudness: -6.6 dB
• mode: 1.000
• speechiness: 0.047
• acousticness: 0.154
• instrumentalness: 0.000
• liveness: 0.124
• valence: 0.451 (45%)
• tempo: 120 BPM
• duration_ms: 3.5 min

2. Avec genre 'acoustic':
🎵 Paramètres optimaux pour le genre: acoustic

📊 PARAMÈTRES OPTIMAUX:
• danceability: 0.582 (58%)
• energy: 0.365 (37%)
• key: 5.000
• loudness: -9.0 dB
• mode: 1.000
• speechiness: 0.034
• acousticness: 0.671
• instrumentalness: 0.000
• liveness: 0.114
• valence: 0.357 (36%)
• tempo: 119 BPM
• duration_ms: 3.7 min

3. Avec genre 'pop':
🎵 Paramètres optimaux pour le genre: pop

📊 PARAMÈTRES OPTIMAUX:
• danceability: 0.664 (66%)
• energy: 0.637 (64%)
• key: 6.000
• loudness: -6.0 dB
• mode: 1.000
• speechiness: 0.051
• acousticness: 0.165
• instrumentalness: 0.000
• liveness: 0.114
• v

In [21]:
import pandas as pd
import os
from dotenv import load_dotenv
import re
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.messages import HumanMessage

load_dotenv()

# Configuration LangChain Mistral
llm = ChatMistralAI(
    model="mistral-small-latest",
    mistral_api_key=os.environ["MISTRAL_API_KEY"]
)

def get_optimal_parameters_with_constraints(genre=None, tempo=None, key=None, duration_min=None):
    df = pd.read_csv('dataset.csv')
    
    # Filtre de base pour les musiques populaires (top 20%)
    top_songs = df[df['popularity'] >= df['popularity'].quantile(0.8)]
    
    # Application des contraintes progressives
    filtered_songs = top_songs.copy()
    
    if genre:
        genre_songs = filtered_songs[filtered_songs['track_genre'] == genre]
        if len(genre_songs) > 10:  # Seulement si on a assez de données
            filtered_songs = genre_songs
    
    if tempo:
        # Recherche des tempos similaires (±10 BPM)
        tempo_range = (tempo - 10, tempo + 10)
        tempo_songs = filtered_songs[
            (filtered_songs['tempo'] >= tempo_range[0]) & 
            (filtered_songs['tempo'] <= tempo_range[1])
        ]
        if len(tempo_songs) > 5:
            filtered_songs = tempo_songs
    
    if key is not None:
        key_songs = filtered_songs[filtered_songs['key'] == key]
        if len(key_songs) > 3:
            filtered_songs = key_songs
    
    if duration_min:
        duration_ms = duration_min * 60000
        duration_range = (duration_ms * 0.7, duration_ms * 1.3)  # ±30%
        duration_songs = filtered_songs[
            (filtered_songs['duration_ms'] >= duration_range[0]) & 
            (filtered_songs['duration_ms'] <= duration_range[1])
        ]
        if len(duration_songs) > 3:
            filtered_songs = duration_songs
    
    # Si trop filtré, on élargit légèrement
    if len(filtered_songs) < 5:
        filtered_songs = top_songs  # Retour au dataset complet
    
    optimal_params = filtered_songs[[
        'danceability', 'energy', 'key', 'loudness', 'mode',
        'speechiness', 'acousticness', 'instrumentalness',
        'liveness', 'valence', 'tempo', 'duration_ms'
    ]].median().to_dict()
    
    return optimal_params, len(filtered_songs)

def analyze_user_request(user_input):
    prompt = f"""
    Analyse cette demande musicale et extrais les informations suivantes au format JSON:
    - genre: le genre musical (ex: "rock", "pop", "jazz") ou null
    - tempo: le tempo en BPM (nombre) ou null
    - key: la tonalité (ex: "C", "D", "Emin", "F#m") ou null
    - duration: la durée en minutes (nombre) ou null
    
    Demande: "{user_input}"
    
    Réponds UNIQUEMENT avec le JSON, rien d'autre.
    """
    
    messages = [HumanMessage(content=prompt)]
    response = llm.invoke(messages)
    
    try:
        # Nettoyage de la réponse
        json_str = response.content.strip()
        json_str = re.sub(r'^```json\s*|\s*```$', '', json_str)  # Retire les backticks
        import json
        constraints = json.loads(json_str)
        return constraints
    except:
        # Fallback simple en cas d'erreur
        return {"genre": None, "tempo": None, "key": None, "duration": None}

def convert_key_to_number(key_str):
    """Convertit une tonalité texte en numéro de key (0-11)"""
    if not key_str:
        return None
    
    key_mapping = {
        'C': 0, 'C#': 1, 'Db': 1, 'D': 2, 'D#': 3, 'Eb': 3,
        'E': 4, 'F': 5, 'F#': 6, 'Gb': 6, 'G': 7, 'G#': 8,
        'Ab': 8, 'A': 9, 'A#': 10, 'Bb': 10, 'B': 11
    }
    
    # Gestion des modes mineurs (ex: "Amin" → "A")
    key_clean = key_str.upper().replace('MIN', '').replace('M', '').replace(' ', '')
    
    for base_key, key_num in key_mapping.items():
        if base_key in key_clean:
            return key_num
    
    return None

def generate_music_advice(constraints, parameters, sample_size):
    # Conversion des paramètres techniques en conseils pratiques
    key_mapping = {
        0: "C", 1: "C#", 2: "D", 3: "D#", 4: "E", 
        5: "F", 6: "F#", 7: "G", 8: "G#", 9: "A", 10: "A#", 11: "B"
    }
    
    mode_text = "majeur" if parameters['mode'] >= 0.5 else "mineur"
    key_text = key_mapping.get(int(parameters['key']), "Inconnue")
    
    # Construction du contexte des contraintes
    constraints_text = []
    if constraints.get('genre'):
        constraints_text.append(f"Genre: {constraints['genre']}")
    if constraints.get('tempo'):
        constraints_text.append(f"Tempo cible: {constraints['tempo']} BPM")
    if constraints.get('key'):
        constraints_text.append(f"Tonalité: {constraints['key']}")
    if constraints.get('duration'):
        constraints_text.append(f"Durée: {constraints['duration']} min")
    
    constraints_context = " | ".join(constraints_text) if constraints_text else "Aucune contrainte spécifique"
    
    # Formatage des paramètres utiles
    useful_params = {
        "Tonalité": f"{key_text} {mode_text}",
        "Tempo": f"{parameters['tempo']:.0f} BPM",
        "Énergie": f"{parameters['energy']*100:.0f}%",
        "Danse": f"{parameters['danceability']*100:.0f}%",
        "Positivité": f"{parameters['valence']*100:.0f}%",
        "Acoustique": f"{parameters['acousticness']*100:.0f}%",
        "Instrumental": f"{parameters['instrumentalness']*100:.1f}%",
        "Durée": f"{parameters['duration_ms']/60000:.1f} min"
    }
    
    params_text = "\n".join([f"• {k}: {v}" for k, v in useful_params.items()])
    
    prompt = f"""
    CONTEXTE: L'utilisateur veut créer une musique avec ces contraintes: {constraints_context}
    Base de données: {sample_size} morceaux similaires analysés
    
    Donne 3 conseils pratiques pour composer avec ces caractéristiques:
    {params_text}
    
    Conseils demandés:
    1. Structure rythmique adaptée au tempo
    2. Progression d'accords pour la tonalité
    3. Ambiance et énergie globale
    4. Équilibre instrumental
    
    Sois concret et donne des exemples musicaux.
    """
    
    messages = [HumanMessage(content=prompt)]
    response = llm.invoke(messages)
    return response.content, useful_params, constraints_context

# Interface utilisateur
user_input = input("🎤 Quelle musique veux-tu créer ? (ex: 'rock à 120 BPM en La mineur de 3 minutes') ")

constraints = analyze_user_request(user_input)

# Conversion des contraintes
tempo_num = int(constraints['tempo']) if constraints.get('tempo') else None
key_num = convert_key_to_number(constraints.get('key'))
duration_num = float(constraints['duration']) if constraints.get('duration') else None

parameters, sample_size = get_optimal_parameters_with_constraints(
    genre=constraints.get('genre'),
    tempo=tempo_num,
    key=key_num,
    duration_min=duration_num
)

advice, useful_params, constraints_context = generate_music_advice(constraints, parameters, sample_size)

# Affichage des résultats
print(f"\n🎵 PARAMÈTRES OPTIMISÉS POUR: {constraints_context}")
print(f"📊 (basé sur {sample_size} morceaux similaires)\n")

for param, value in useful_params.items():
    print(f"{param}: {value}")

print(f"\n💡 CONSEILS DE COMPOSITION:\n{advice}")

# Suggestions alternatives si beaucoup de contraintes
if sample_size < 20 and (tempo_num or key_num or duration_num):
    print(f"\n💡 INFO: Peu de données avec toutes ces contraintes. Essayez de relâcher certaines spécifications pour plus de variété!")


🎵 PARAMÈTRES OPTIMISÉS POUR: Genre: pop
📊 (basé sur 644 morceaux similaires)

Tonalité: F majeur
Tempo: 111 BPM
Énergie: 61%
Danse: 65%
Positivité: 49%
Acoustique: 34%
Instrumental: 0.0%
Durée: 3.7 min

💡 CONSEILS DE COMPOSITION:
Voici 3 conseils pratiques pour composer une pop énergique en **F majeur (111 BPM)** avec les caractéristiques analysées :

---

### **1. Structure rythmique adaptée au tempo (111 BPM)**
- **Exemple concret** : Utilise un **groove pop mid-tempo** avec une **pulsation claire** (ex: batterie avec caisse claire sur les temps 2 et 4, comme dans *"Levitating" – Dua Lipa*).
- **Variantes** :
  - **Pre-chorus** : Ralentir légèrement (90 BPM) pour créer un effet de tension avant le refrain.
  - **Breakdown** : Ajouter des **syncopations** (ex: hi-hats décalés) pour dynamiser sans accélérer le tempo.

---

### **2. Progression d'accords pour la tonalité (F majeur)**
- **Exemple de progression** :
  **F – C – Dm – Bb** (classique en pop, comme *"Can't Stop the Feeling!